In [21]:
import pandas as pd
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import os
import shutil
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import LabelBinarizer
import pickle

In [22]:
def time_start(name_file):
    file_tsv = name_file[:-3]+'tsv'
    # Đọc tệp TSV bằng pandas
    # print(file_tsv)
    df = pd.read_csv(file_tsv, delimiter='\t')
    df=df.iloc
    df0 = df[:,0]
    df2 = df[:,2]
    for i in range(len(df2)):
        if(df2[i]==1):
            time = df0[i]
            break
    return df0[i]

In [23]:
### Chia file.wav ra moi folder

# data = 'training_data'

# out_file = 'training_data_all/'
# for file in os.listdir(data):
#     if(file.endswith(".wav") == True):

#         name_file = os.path.join(data,file)
#         time_begin = (time_start(name_file))
#         file_txt = (name_file.split('_')[0])+'_'+(name_file.split('_')[1])+'.txt'
#         # print(file_txt)
#         df = pd.read_csv(file_txt)
#         df=np.array(df)
#         # print(df)
#         if (df == '#Murmur: Absent').any().any():
#             # print('Absent')
#             shutil.copy(name_file, out_file +'Absent/'+str(time_begin)+'_'+file)
#         elif (df == '#Murmur: Present').any().any():
#             # print('Present')
#             shutil.copy(name_file, out_file +'Present/'+str(time_begin)+'_'+file)
#         elif (df == '#Murmur: Unknown').any().any():
#             # print('Unknown')
#             shutil.copy(name_file, out_file +'Unknown/'+str(time_begin)+'_'+file)

In [24]:
""" 
/training_data_all
        --->Unknown
        --->Absent
        --->Present
"""

' \n/training_data_all\n        --->Unknown\n        --->Absent\n        --->Present\n'

In [25]:
def show_spectrogram (file):
    audio, sr = librosa.load(file,sr=4000)
    mel_spectrogram = librosa.feature.melspectrogram(y=audio, sr=sr,n_fft= 200 ,n_mels=64,hop_length=300)
    mel_spectrogram_db = librosa.power_to_db(mel_spectrogram ,ref=np.max)

    fig, ax = plt.subplots(figsize=(10,4))
    a =librosa.display.specshow(mel_spectrogram_db,
                            x_axis='time',
                            y_axis='linear',
                            sr=sr,
                            hop_length=512,
                            )
    plt.title("Audio Spectrogram Representation")
    plt.colorbar(format='%+2.0f dB')
    plt.xlabel("Time(s)")
    plt.ylabel("Frequency (Hz)")
    plt.show()


In [26]:
# audio_file_absent = "training_data_all/Absent/0.02025_49745_TV.wav"
# audio_file_present = "training_data_all/Present/0.08025_85031_AV.wav"
# audio_file_unknown = "training_data_all/Unknown/0.720351_49761_MV.wav"
# show_spectrogram(audio_file_absent)
# show_spectrogram(audio_file_present)
# show_spectrogram(audio_file_unknown)

In [27]:
data_train = 'training_data_all'
data_time = []
for folders in os.listdir(data_train):
    print(folders)
    name_folders = os.path.join(data_train,folders)
    for file in os.listdir(name_folders):
        name_files = os.path.join(name_folders,file)
        y,sr = librosa.load(name_files,sr=4000)
        audio_duration = (len(y)/sr)
        data_time.append(audio_duration)

Unknown
Absent
Present


In [28]:
np.min(data_time)


5.152

In [45]:
def create_data (audio,file,sr):
    mel_spectrogram = librosa.feature.melspectrogram(y=audio, sr=sr,n_fft= 256 ,n_mels=128,hop_length=157)
    mel_spectrogram_db = librosa.power_to_db(mel_spectrogram ,ref=np.max)
    # mel_spectrogram_db = mel_spectrogram_db.reshape([128,128,1])
    x_data.append(mel_spectrogram_db)


In [46]:
def split_audio(name_files,file,time_limit,sr):
    audio, sr = librosa.load(name_files, sr=sr)
    file = file[:-4]
    segment_length = time_limit * sr
    num_segments = (len(audio) // segment_length)
    for i in range(num_segments):
        start_sample = i * segment_length
        end_sample = (i + 1) * segment_length

        segment = audio[start_sample:end_sample]
        create_data(segment,file,sr)
        label = name_files.split('/')[1]
        get_label(label)

In [47]:
def get_label(label):
    # print(folders)
    y_data.append(label)

In [48]:
def get_date(name_files,file,time_limit,sr = 4000):
    # print(name_files)
    split_audio(name_files,file,time_limit,sr)

In [49]:
data_train = 'training_data_all'
x_data = []
y_data = []
for folders in os.listdir(data_train):
    print(folders)
    name_folders = os.path.join(data_train,folders)
    for file in os.listdir(name_folders):
        name_files = os.path.join(name_folders,file)
        get_date(name_files,file,time_limit = 5,sr = 4000)


file = open('data_file.csv','wb')
pickle.dump((x_data,y_data),file)
file.close()

Unknown
Absent
Present


In [50]:
file = open('data_file.csv', 'rb')
data = pickle.load(file)
x_data,y_data =data
file.close()
x_data=np.array(x_data)
y_data = np.array(y_data)

In [51]:
from collections import Counter

counters = Counter(y_data)

for value, count in counters.items():
    print(f"{value} xuất hiện {count}")


Unknown xuất hiện 622
Absent xuất hiện 9865
Present xuất hiện 2453


In [52]:
x_data=np.array(x_data)
y_data=np.array(y_data)
print(x_data.shape)
print(y_data.shape)

(12940, 128, 128)
(12940,)


In [53]:
x_test = []
y_test = []
x_train = []
y_train = []

In [54]:
encoder = LabelBinarizer()
y_data=encoder.fit_transform(y_data)
x_train,x_test,y_train,y_test = train_test_split(x_data,y_data,test_size=0.3,random_state=68)

In [55]:
# x_test = np.array(x_test)
# y_test = np.array(y_test)
# x_train = np.array(x_train)
# y_train = np.array(y_train)

In [56]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, BatchNormalization, LSTM, Dense, Dropout

input_shape = (128, 128)


model = Sequential()

model.add(Conv1D(64, kernel_size=3, strides=1, padding='same', activation='relu', input_shape=input_shape))
model.add(MaxPooling1D(pool_size=2, strides = 2, padding = 'same'))
model.add(BatchNormalization())

model.add(Conv1D(128, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2, strides = 2, padding = 'same'))
model.add(BatchNormalization())

model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2, strides = 2, padding = 'same'))
model.add(BatchNormalization())

model.add(LSTM(128,return_sequences=True))
model.add(LSTM(64, return_sequences=False))  


model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(3, activation='softmax')) 


model.compile(optimizer='adam', loss='MSE', metrics=['accuracy'])

model.summary()


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_15 (Conv1D)          (None, 128, 64)           24640     
                                                                 
 max_pooling1d_15 (MaxPooli  (None, 64, 64)            0         
 ng1D)                                                           
                                                                 
 batch_normalization_15 (Ba  (None, 64, 64)            256       
 tchNormalization)                                               
                                                                 
 conv1d_16 (Conv1D)          (None, 64, 128)           24704     
                                                                 
 max_pooling1d_16 (MaxPooli  (None, 32, 128)           0         
 ng1D)                                                           
                                                      

In [58]:
history = model.fit(x_train,y_train,batch_size=32, epochs=10,validation_data=(x_test, y_test), callbacks=[model_checkpoint])

Epoch 1/10
284/284 [==============================] - 3s 11ms/step - loss: 0.1056 - accuracy: 0.8042 - val_loss: 0.1175 - val_accuracy: 0.7893
Epoch 2/10
284/284 [==============================] - 3s 10ms/step - loss: 0.1047 - accuracy: 0.8074 - val_loss: 0.1043 - val_accuracy: 0.8073
Epoch 3/10
284/284 [==============================] - 3s 10ms/step - loss: 0.1041 - accuracy: 0.8087 - val_loss: 0.1129 - val_accuracy: 0.7960
Epoch 4/10
284/284 [==============================] - 3s 10ms/step - loss: 0.1023 - accuracy: 0.8119 - val_loss: 0.1093 - val_accuracy: 0.7962
Epoch 5/10
284/284 [==============================] - 3s 10ms/step - loss: 0.1010 - accuracy: 0.8159 - val_loss: 0.1092 - val_accuracy: 0.7983
Epoch 6/10
284/284 [==============================] - 3s 10ms/step - loss: 0.1028 - accuracy: 0.8103 - val_loss: 0.1223 - val_accuracy: 0.7751
Epoch 7/10
284/284 [==============================] - 3s 10ms/step - loss: 0.0995 - accuracy: 0.8166 - val_loss: 0.1071 - val_accuracy: 0.8009